In [1]:
import nltk
import pandas as pd
import numpy as np
import numbers
from num2words import num2words
import collections
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk.collocations
import nltk.corpus
import collections
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet
from nltk.tag.stanford import StanfordPOSTagger

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\ACASA-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\ACASA-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\ACASA-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def preprocessing(data):

    from nltk.corpus import stopwords
    lemmatizer = WordNetLemmatizer()
    
    translate_table = dict((ord(char), None) for char in string.punctuation) 
    data1 = data.translate(translate_table)
    data2 = word_tokenize(data1)

    data2 = [w.lower() for w in data2]

    stopwords = set(stopwords.words('english'))
    data3= [x for x in data2 if not x in stopwords]  

    preprocessed_data  = []
    for word in data3:
        preprocessed_data.append(lemmatizer.lemmatize(word))

    return preprocessed_data

In [3]:
def compare_glosses(gloss1, gloss2):
    
    gloss_1 = preprocessing(gloss1)
    gloss_2 = preprocessing(gloss2)

    score = 0
    for word1 in gloss_1:
        for word2 in gloss_2:
            if word1 == word2:
                score += 1

    return score

In [4]:
def Original_Lesk(context, word, pos_tag):
    
    maximum_score = 0
    result = None
    
    for word_sense in wordnet.synsets(word):
        if word_sense.pos() == pos_tag:
            sense_score = 0

            for cont in context:    
                for context_sense in wordnet.synsets(cont):
                    if context_sense.pos() == pos_tag:
                        sense_score += len(set(preprocessing(word_sense.definition())) & set(preprocessing(context_sense.definition())))

            if sense_score > maximum_score:
                maximum_score = sense_score
                result = word_sense
            
    return result

In [5]:
Original_Lesk(nltk.word_tokenize('Students enjoy going to school, studying and reading books'), 'school', 'n')

Synset('school.n.04')

In [7]:
from nltk.wsd import lesk
lesk(nltk.word_tokenize('Students enjoy going to school, studying and reading books'),'school','n')

Synset('school.n.06')

In [8]:
syns=wordnet.synset('music.n.04')
syns.definition()

'(music) the sounds produced by singers or musical instruments (or reproductions of such sounds)'

# Extended Lesk

In [9]:
def glosses_of_synsets(synset):
    

    hypernyms = ''
    for hypernym in synset.hypernyms():
        hypernyms += hypernym.definition()
        
    hyponyms = ''
    for hyponym in synset.hyponyms():
        hyponyms += hyponym.definition()
    
    holonyms = ''
    for holonym in synset.substance_holonyms() + synset.part_holonyms() + synset.part_holonyms():
        holonyms += holonym.definition()
    
    meronyms = ''
    for meronym in synset.substance_meronyms() + synset.part_meronyms() + synset.part_meronyms():
        meronyms += meronym.definition()
    
    troponyms = ''
    for troponym in synset.entailments():
        troponyms += troponym.definition()
    
    attributes = ''
    for attribute in synset.attributes():
        attributes += attribute.definition()
    
    similars = ''
    for similar in synset.similar_tos():
        similars += similar.definition()
    
    alsos = ''
    for also in synset.also_sees():
        alsos += also.definition()

    return [hypernyms,hyponyms,holonyms,meronyms,troponyms,attributes,similars,alsos]


In [10]:
glosses_of_synsets(wordnet.synset('air.n.01'))

['a fluid in the gaseous state having neither independent shape nor volume and being able to expand indefinitely',
 'the air that is inhaled and exhaled in respirationair that has been heated and tends to riseair in a liquid state',
 'air moving (sometimes with considerable force) from an area of high pressure to an area of low pressure',
 "a colorless and odorless inert gas; one of the six inert gases; comprises approximately 1% of the earth's atmospherea colorless element that is one of the six inert gasses; occurs in trace amounts in aira colorless odorless gaseous element that give a red glow in a vacuum tube; one of the six inert gasses; occurs in the air in small amountsa common nonmetallic element that is normally a colorless odorless tasteless inert diatomic gas; constitutes 78 percent of the atmosphere by volume; a constituent of all living tissuesa nonmetallic bivalent element that is normally a colorless odorless tasteless nonflammable diatomic gas; constitutes 21 percent of